<a href="https://colab.research.google.com/github/SouvikakaPuka/Data-Science/blob/master/MovieReviewsKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import the pandas library to read our dataset
import pandas as pd
from pandas import DataFrame, Series
# Get the train/test split package from sklearn for preparing our dataset to
# train and test the model with
from sklearn.model_selection import train_test_split
#For PCA
from sklearn.decomposition import PCA, TruncatedSVD
# Import the numpy library to work with and manipulate the data
import numpy as np
import nltk
import random
nltk.download('punkt')
nltk.download('stopwords')
# nltk.download('movie_reviews')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, GlobalAveragePooling1D, BatchNormalization, Dropout
from keras.optimizers import SGD, Adam, Nadam, RMSprop, Adamax
from keras.utils import to_categorical

Using TensorFlow backend.


In [3]:
dataset = pd.read_csv('https://raw.githubusercontent.com/cacoderquan/Sentiment-Analysis-on-the-Rotten-Tomatoes-movie-review-dataset/master/train.tsv', sep = '\t')
dataset.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [0]:
# X = dataset['Phrase']
# Y = dataset['Sentiment']
# print(X.shape, Y.shape)

In [5]:
ds = dataset.copy()
ds = ds.drop(columns = ['PhraseId', 'SentenceId'])
ds.head()

,Phrase,Sentiment
0,A series of escapades demonstrating the adage ...,1
1,A series of escapades demonstrating the adage ...,2
2,A series,2
3,A,2
4,series,2


In [0]:
documents = ds.values.tolist()

In [7]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer
porter = PorterStemmer()
lancaster=LancasterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()
stopwords_en = stopwords.words("english")
punctuations="?:!.,;'\"-()"
#parameters to adjust to see the impact on outcome
remove_stopwords = True
useStemming = False
useLemma = True
removePuncs = True

for l in range(len(documents)): #For each review document
  label = documents[l][1] #Save review label
  tmpReview = [] #Placeholder list for new review
  for w in documents[l][0].split(' '): #For each word this is review
    newWord = w #Set newWork to be the updated word
    if remove_stopwords and (w in stopwords_en):#if the word is a stopword & we want to remove stopwords
      continue #skip the word and don’t had it to the normalized review
    if removePuncs and (w in punctuations):#if the word is a punc. & we want to remove punctuations
      continue #skip the word and don’t had it to the normalized review
    if useStemming: #if useStemming is set to True
      #Keep one stemmer commented out
      #newWord = porter.stem(newWord) #User porter stemmer
      newWord = lancaster.stem(newWord) #Use Lancaster stemmer
    if useLemma:
      newWord = wordnet_lemmatizer.lemmatize(newWord)
    tmpReview.append(newWord) #Add normalized word to the tmp review
  documents[l] = (' '.join(tmpReview), label) #Update the reviews list with clean review
print(documents[1])

('A series escapade demonstrating adage good goose', 2)


In [0]:
all_data = pd.DataFrame(documents,
                                columns=['text', 'sentiment'])
# Splits the dataset so 70% is used for training and 30% for testing
X_train, X_test, Y_train, Y_test = train_test_split(all_data['text'], all_data['sentiment'], test_size=0.3)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#Initialize number of features to be considered
vec_length = 2500
# Transform each text into a vector of word counts
# vectorizer = CountVectorizer(stop_words="english",
#                              ngram_range=(1, 1), max_features = vec_length)
vectorizer = TfidfVectorizer(stop_words="english",
                            ngram_range=(1, 1), max_features = vec_length)
#X = vectorizer.fit_transform(all_data["text"])
#Y = all_data['sentiment']
x_train = vectorizer.fit_transform(X_train)
y_train = Y_train
x_test = vectorizer.transform(X_test)
y_test = Y_test

In [10]:
# Converts the datasets to numpy arrays to work with the model

np.random.seed(9001)

#Convert the training data
# x_train_np = x_train
x_train_np = x_train.toarray()
y_train_np = to_categorical(y_train)
# y_train_np = np.array(y_train)

# Convert the testing data
# x_test_np = x_test
x_test_np = x_test.toarray()
y_test_np = to_categorical(y_test)
# y_test_np = np.array(y_test)

print(x_train_np.shape)
print(y_train_np.shape)
print(x_test_np.shape)
print(y_test_np.shape)

(109242, 2500)
(109242, 5)
(46818, 2500)
(46818, 5)


In [0]:
# Changing input variable distribution to format to feed the model

x_train_np = np.expand_dims(x_train_np, axis=2)
x_test_np = np.expand_dims(x_test_np, axis=2)

In [12]:
#create model

model = Sequential()
model.add(BatchNormalization())
model.add(Conv1D(filters = 128, kernel_size=1, activation='relu', input_shape=(x_train_np.shape[1],1)))
# model.add(MaxPooling1D(pool_size =2))
model.add(Conv1D(filters = 128, kernel_size=1, activation='relu'))
# model.add(MaxPooling1D(pool_size =2))
# model.add(Conv1D(filters = 128, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size =2))
# model.add(GlobalAveragePooling1D())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(BatchNormalization())
model.add(Dense(5, activation='softmax'))

In [0]:
# Create metrics for Precision, Recall, and F-1 Score

from keras import backend as K

def recall(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  recall = true_positives / (possible_positives + K.epsilon())
  return recall

def precision(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  return precision

def f1score(y_true, y_pred):

  prec = precision(y_true, y_pred)
  rec = recall(y_true, y_pred)
  f1score = (2 * (prec * rec)/(rec + prec))
  return f1score


In [0]:
# Optimizers
adam = keras.optimizers.Adam(lr = 0.0001)
sgd = keras.optimizers.SGD(lr = 0.0001)
nadam = keras.optimizers.Nadam(lr = 0.0001)
rms = keras.optimizers.RMSprop(lr = 0.0001)
adamax = keras.optimizers.Adamax(lr = 0.0001)
#Compiling the model
model.compile(optimizer=adamax, loss='categorical_crossentropy', metrics=['accuracy', recall, precision, f1score])

In [22]:
batch_size = 128  # Batch size
epochs = 10 # Number of eopchs

#Training the model
history = model.fit(x_train_np, y_train_np, validation_data=( x_test_np, y_test_np), epochs= epochs, batch_size = batch_size)

Train on 109242 samples, validate on 46818 samples
Epoch 1/10
109242/109242 [==============================] - 123s 1ms/step - loss: 0.6280 - acc: 0.7498 - recall: 0.7224 - precision: 0.7710 - f1score: 0.7458 - val_loss: 1.2116 - val_acc: 0.6132 - val_recall: 0.5837 - val_precision: 0.6301 - val_f1score: 0.6059
Epoch 2/10
109242/109242 [==============================] - 123s 1ms/step - loss: 0.6216 - acc: 0.7512 - recall: 0.7243 - precision: 0.7725 - f1score: 0.7475 - val_loss: 1.2183 - val_acc: 0.6142 - val_recall: 0.5875 - val_precision: 0.6290 - val_f1score: 0.6074
Epoch 3/10
109242/109242 [==============================] - 122s 1ms/step - loss: 0.6193 - acc: 0.7513 - recall: 0.7243 - precision: 0.7727 - f1score: 0.7476 - val_loss: 1.2350 - val_acc: 0.6141 - val_recall: 0.5843 - val_precision: 0.6284 - val_f1score: 0.6055
Epoch 4/10
109242/109242 [==============================] - 121s 1ms/step - loss: 0.6177 - acc: 0.7520 - recall: 0.7253 - precision: 0.7727 - f1score: 0.7482 - val

In [23]:
#Evaluating the model on training data

train_loss, train_accuracy, train_recall, train_precision, train_f1score = model.evaluate(x_train_np, y_train_np)
print("Train Loss: ", train_loss)
print("Train Accuracy: ", train_accuracy)
print("Train Recall: ", train_recall)
print("Train Precision: ", train_precision)
print("Train F1 Score: ", train_f1score)

109242/109242 [==============================] - 48s 442us/step
Train Loss:  0.5905579630968822
Train Accuracy:  0.7626645429359034
Train Recall:  0.7379487742765892
Train Precision:  0.7825544319272257
Train F1 Score:  0.7592325137338142


In [24]:
#Evaluating the model on test data

test_loss, test_accuracy, test_recall, test_precision, test_f1score = model.evaluate(x_test_np, y_test_np)
print("Test Loss: ", test_loss)
print("Test Accuracy: ", test_accuracy)
print("Test Recall: ", test_recall)
print("Test Precision: ", test_precision)
print("Test F1 Score: ", test_f1score)

46818/46818 [==============================] - 20s 430us/step
Test Loss:  1.2744478743773686
Test Accuracy:  0.6124994660173437
Test Recall:  0.5881498568926481
Test Precision:  0.6253334948648794
Test F1 Score:  0.6058717137180739


In [0]:
model.save('1116613_1dconv_classify.h5')